In [1]:
from core import init_langchain_cache, init_requests_cache

init_langchain_cache()
init_requests_cache()

'/home/keith/company-detective/.cache/requests_cache.sqlite'

In [2]:
from typing import NamedTuple, Mapping, List

from core import CompanyProduct
from search import SearchResult
import news.search
import news.scrape
import news.summarize

class NewsSummary(NamedTuple):
    # input
    target: CompanyProduct

    # intermediates
    search_results: List[SearchResult]
    article_markdowns: Mapping[str, str]

    # output
    summary_markdown: str

    @property
    def summary_html(self):
        # Hack to make the header levels work in the larger context
        # TODO: Refactor this
        cleaned_markdown = re.sub(r"^#", "###", self.summary_markdown, flags=re.MULTILINE)

        # Add a newline before bulleted lists in markdown
        cleaned_markdown = re.sub(r"^([^-].*)\n-", r"\1\n\n-", cleaned_markdown, flags=re.MULTILINE)


        html_out = markdown.markdown(cleaned_markdown)

        return html_out


def process_news(target: CompanyProduct) -> NewsSummary:
    search_results = news.search.find_news_articles(target, num_results=30)

    article_markdowns = {result.link: news.scrape.get_article_markdown(result.link) for result in search_results}

    article_markdown_list = [article for article in article_markdowns.values() if article]

    llm_result = news.summarize.summarize(target, article_markdown_list)

    return NewsSummary(
        target=target, 
        search_results=search_results, 
        article_markdowns=article_markdowns, 
        summary_markdown=llm_result.content
        )

target = CompanyProduct.same("98point6")
news_result = process_news(target)


Failed to get article from https://www.geekwire.com/2024/98point6-hit-by-new-layoffs-in-latest-change-at-health-tech-startup/: 403
Failed to get article from https://notice.co/c/98point6: 403
Failed to get article from https://www.reuters.com/article/brief-98point6-inc-files-to-say-raised-2/brief-98point6-inc-files-to-say-raised-29-1-million-in-equity-financing-idUSFWN1M10QV/: 401
Failed to get article from https://www.axios.com/pro/health-tech-deals/newsletters/2024/01/31/health-tech-redesigns-pale-portfolio: 403
Failed to get article from https://www.crunchbase.com/organization/mdlive/company_overview/overview_timeline: 403
Timeout on https://ensignbenefits.com/benefit-basics/new-hire-enrollment
Failed to get article from https://ptdiocese.org/benefits: 403
37,985 characters in unified context


NameError: name 'pprint' is not defined

In [3]:
from pprint import pprint
pprint(news_result.summary_markdown)

('# Summary of 98point6 Technologies\n'
 '\n'
 '## Company Overview\n'
 '98point6 Technologies is a Seattle-based health tech company that '
 'specializes in licensed on-demand virtual care software. Founded in 2015, '
 'the company initially operated as a virtual primary care provider but has '
 'since pivoted to focus on licensing its technology to healthcare providers. '
 'The flagship product, the 98point6 Technology Platform, is a cloud-based '
 'software solution that integrates artificial intelligence with automated '
 'clinical standards to enhance virtual care delivery.\n'
 '\n'
 '## Key Developments\n'
 '\n'
 '### Acquisitions\n'
 '- **Bright.md**: On January 16, 2024, 98point6 announced the acquisition of '
 "Bright.md's remaining assets, which includes 16 customer contracts and the "
 'sales and customer success teams. This acquisition is part of a strategy to '
 'enhance its asynchronous care capabilities, allowing healthcare providers to '
 'communicate with patients at t

In [4]:
news_result.search_results

[SearchResult(title='98point6 Technologies Announces the Acquisition of Bright.md to ...', link='https://www.prnewswire.com/news-releases/98point6-technologies-announces-the-acquisition-of-brightmd-to-accelerate-the-launch-of-its-asynchronous-care-module-302034295.html', snippet='Jan 16, 2024 ... SEATTLE, Jan. 16, 2024 /PRNewswire/ -- 98point6 Technologies, a leader in licensed on-demand virtual care software, announced the addition of a new asynchronous\xa0...', formattedUrl='https://www.prnewswire.com/news.../98point6-technologies-announces-the-...'),
 SearchResult(title='98point6 hit by new layoffs in latest change at health tech startup ...', link='https://www.geekwire.com/2024/98point6-hit-by-new-layoffs-in-latest-change-at-health-tech-startup/', snippet='Apr 23, 2024 ... In March of last year, 98point6 announced that it was selling its virtual care platform and primary care business to Transcarent for $100 million in cash and\xa0...', formattedUrl='https://www.geekwire.com/.../98

In [5]:
news_result_cached = process_news(target)
pprint(news_result_cached.summary_markdown)

37,985 characters in unified context
('# Summary of 98point6 Technologies\n'
 '\n'
 '## Company Overview\n'
 '98point6 Technologies is a Seattle-based health tech company that '
 'specializes in licensed on-demand virtual care software. Founded in 2015, '
 'the company initially operated as a virtual primary care provider but has '
 'since pivoted to focus on licensing its technology to healthcare providers. '
 'The flagship product, the 98point6 Technology Platform, is a cloud-based '
 'software solution that integrates artificial intelligence with automated '
 'clinical standards to enhance virtual care delivery.\n'
 '\n'
 '## Key Developments\n'
 '\n'
 '### Acquisitions\n'
 '- **Bright.md**: On January 16, 2024, 98point6 announced the acquisition of '
 "Bright.md's remaining assets, which includes 16 customer contracts and the "
 'sales and customer success teams. This acquisition is part of a strategy to '
 'enhance its asynchronous care capabilities, allowing healthcare providers 